# Proyecto Bootcamp DS - Codigo Facilito
## Predicción y Analisis de Rotacion en una empresa bancaria
### Indice
0. Recolectando Data
1. Contexto del problema
2. Analisis Actual
3. Featuring Engineering
4. Modelado
5. Evaluacion del modelo
6. Resultados gráficos

In [1]:
import pandas as pd
import numpy as np
import openpyxl 
import altair as alt

* Obtengo Ceses de Entrrevistas de salida desde el 2017
    * Con esto puedo ver que la mayor cantidad de ceses en nro de personas esta centradas en negocios Individual y Grupales  

In [2]:
ceses = pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/Entrevistas de Salida.xlsx',
    usecols='E,I,O,P,S,T,W,X',
    dtype={'CODIGO':str})

C:\Users\Anna\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


* Tranformaciones necesarias en el dataset
    * Asumiendo que se quiere analizar solo la fuerza de ventas (invidual y grupal) por eso filtro solo estos para el analisis
    * Solo utilizare ceses desde el 2018, porque el resto de la info es desde este ano, ademas por el evento de la mudanza donde hubieron muchos ceses atipicos
    * Transformar a mes la fecha de cese porque es el intervalo que me interesa evaluar

In [3]:
ceses['CESE'] = ceses['CESE'].to_numpy().astype('datetime64[M]')

ceses = ceses[
    (ceses['CESE']>= '2018-01-01') &
    (ceses['Producto'].isin(['INDIVIDUAL', 'GRUPAL']))]

* Quiero ver la cantidad de cesados por propoducto para darme una idea

In [4]:
ceses_product = ceses.groupby(
    by=['Producto','CESE'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count')
    )
# plot por producto
alt.Chart(ceses_product).mark_bar().encode(
y = 'year(CESE):N',
x = 'sum(cesados)',
color = 'Producto'
) #grupal e invidual por tener mas rotacion FV

alt.Chart(...)

* Extraigo headcount historico para optener informacion de rotacion (cuanto representa esa cantida de ceses del total de peronas)
* Tengo que hallar la rotacion mensual y acumulada la año
* El headcount se tiene desde el 2018, no del 2017, sera necesario filtrar las entrevistas de salida (ademas del evento de la mudanza donde se disparan los ceses)

In [5]:
# llamo al headcount
headcount = pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/PERSONAL.xlsx',
    usecols='A,B,D:H',
    dtype={'COD':str})

* Filtro individual y grupal que es lo que me interesa

In [6]:
headcount = headcount[
    headcount['PRODUCTO'].isin(['INDIVIDUAL', 'GRUPAL'])]

* Agrupo el headcount para tenerlo por producto y por mes

In [7]:
head_prod = headcount.groupby(
    by=['MES','PRODUCTO'],
    as_index = False).agg(real = ('COD', 'count'))

* Join de Ceses y Headcount para hallar la rotacion
    * Hallo la rotacion, multiplico por 100 y redondeo a 1
    * transformo los meses a year para grupar y teneun total de la suma de meses

In [8]:
rot_producto = pd.merge(
    head_prod,
    ceses_product,
    left_on=['MES','PRODUCTO'],
    right_on=['CESE','Producto'])

rot_producto['rotacion'] = ((rot_producto['cesados'] / rot_producto['real'])*100).round(1)
rot_producto['year'] = rot_producto['MES'].to_numpy().astype('datetime64[Y]')

* Grafico la rotacion acumulada anual por producto
    * Se ve que en grupal se tenia mayor rotacion
    * Despues de pandemia las rotaciones han bajado

In [9]:
rot_producto = rot_producto.groupby(
    by = ['year','PRODUCTO'],
    as_index=False
    ).agg(
        rotacion = ('rotacion','sum')
)

barras = alt.Chart(rot_producto).mark_bar().encode(
    y = 'year(year):N',
    x = 'rotacion'
)

texto = barras.mark_text(
    color = 'white',
    align = 'right'
).encode(
    text = alt.Text(
        'rotacion')
)

(barras+texto).facet(
    'PRODUCTO',
    columns = 2
)

alt.FacetChart(...)

* Es necesario saber que cargos de estos productos voy a tomar
    * Ago un analisis de los cargos que tiene mayor rotacion

In [10]:
## Ceses
ceses_cargo = ceses.groupby(
    by=['Producto', 'CARGO', 'CESE'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count')
    )
## Headcount
head_cargo = headcount.groupby(
    by=['PRODUCTO', 'CARGO', 'MES'],
    as_index = False
    ).agg(
        real = ('COD', 'count')
    )
## Join y rotacion
rot_cargo = pd.merge(
    head_cargo,
    ceses_cargo,
    left_on=['MES', 'PRODUCTO', 'CARGO'],
    right_on=['CESE', 'Producto', 'CARGO'])

rot_cargo['rotacion'] = np.where(
    rot_cargo['real'] == 0,
    0,
    ((rot_cargo['cesados'] / rot_cargo['real'])*100).round(1)
    )
    
rot_cargo['year'] = rot_cargo['MES'].to_numpy().astype('datetime64[Y]')

* Analizo en cantidad de ceses cuantos hay por cargo en cada producto
    * Resalta a simple vista Asesores de Credito

In [11]:
alt.Chart(ceses_cargo).mark_bar().encode(
x = 'year(CESE):N',
y = 'sum(cesados):Q',
color = alt.Color(
    'CARGO:N',
    scale= alt.Scale(scheme='category20')),
column = 'Producto'
)

alt.Chart(...)

* Para verificar mi analisis hallo la rotacion anual acumulada por cargo - grupal
    * Gerentes divisionales tiene mayor rotacion, pero como vimos antes tienen poca cantidad de personas
    * Asesores de negocio son los segundos que tiene mayor rotacion y, como vimos antes, en cantidad de cesados son la mayoria de grupal

In [12]:
## group by anual GRUPAL
rot_cargo_g = rot_cargo[
    rot_cargo['PRODUCTO']=='GRUPAL'].groupby(
        by = ['year', 'PRODUCTO', 'CARGO'],
        as_index=False
        ).agg(
            rotacion = ('rotacion','sum')
        )
## Grafico
barras = alt.Chart(rot_cargo_g).mark_bar().encode(
    y = 'year(year):N',
    x = 'rotacion:Q'
)
texto = barras.mark_text(
    color = 'white',
    align = 'right'
).encode(
    text = alt.Text(
        'rotacion')
)

(barras+texto).facet(
    'CARGO',
    columns = 2
)

alt.FacetChart(...)

* Hallo la rotacion de cada cargo en el caso de INDIVIDUAL
    * Se ve ue Gerentes Divisionales y Gerentes Regionales son los que tienen mas rotacion en algunos anos; sin embargo, estos cargos no representan mucho cese en cantidad
    * Tambien esta el cargo de Asesor Negocios, este tiene una alta rotacion y en cantidad de colabores cesados es el que resalta mas

In [13]:
## group by anual INDIVIDUAL   
rot_cargo_i = rot_cargo[
    rot_cargo['PRODUCTO']=='INDIVIDUAL'].groupby(
        by = ['year', 'PRODUCTO', 'CARGO'],
        as_index=False
        ).agg(
            rotacion = ('rotacion','sum')
        )
## Grafico
barras = alt.Chart(rot_cargo_i).mark_bar().encode(
    y = 'year(year):N',
    x = 'rotacion:Q'
)
texto = barras.mark_text(
    color = 'white',
    align = 'right'
).encode(
    text = alt.Text(
        'rotacion')
)

(barras+texto).facet(
    'CARGO',
    columns = 2
)

alt.FacetChart(...)

* Analizando solo Asesor Negocios de Individual y Grupal
* Empiezo analizar los tipos de Cese VOLUNTARIOS O INVOLUNTARIOS en general de todos los ano
    * Siempre hay mas ceses voluntarios en ambos productos
    * Los ceses involuntarios pueden deberse a un mal reclutamiento o definicion del perfil, mala capacitacion por mal desempeno o faltas cometidas (seria otro proyecto a parte)
    * Los ceses voluntarios pueden ser por mejor oferta u otro tipo que no tenga que ver con su desempeno necesariamente (es el proyecto en el que nos vamos a enfocar)

In [14]:
ceses_tipo = ceses[ceses['CARGO'].isin(
    ['ASESOR NEGOCIOS CREDITO GRUPAL',
    'ASESOR NEGOCIOS CREDITO INDIVIDUAL'])].groupby(
    by=['CESE','TIPO', 'Producto'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count')
    )

alt.Chart(ceses_tipo).mark_bar().encode(
    x = 'year(CESE):N',
    y = 'sum(cesados)',
    color = 'TIPO',
    column = 'Producto'
)

alt.Chart(...)

* Tambien es necesario analizar el otro tipo de CESE basado en el tiempo de parmanencia en el puesto antes de cesar
* Con el histograma se muestra el cambio del comportamiento desde el mes 3, por lo que tomaremos la rotacion voluntaria de asesores desde el mes 3
* En el caso de Individual, a partir del mes 21 se ve una bajada constante  de los ceses segun le tiempo. Del mes 12 al 21 aumenta segun el tiempo, pero del 3 al 12 baja segun el tiempo
* *preguntar porque temprana esta definido desde 6 meses*

In [15]:
ceses['INGRESO'] = ceses['INGRESO'].to_numpy().astype('datetime64[M]')
ceses['antiguedad'] = (((ceses['CESE'] - ceses['INGRESO']).dt.days)/30).round(0)

ceses_antg = ceses[(
    ceses['CARGO'].isin(
        ['ASESOR NEGOCIOS CREDITO GRUPAL',
        'ASESOR NEGOCIOS CREDITO INDIVIDUAL'])) &
    (ceses['TIPO'] == 'VOLUNTARIA')].groupby(
    by=['Producto','antiguedad'],
    as_index=False
).agg(cesados = ('CODIGO', 'count'))



In [21]:

# GRAFICANDO
alt.Chart(ceses_antg).mark_bar().encode(
    x = alt.X(
        'antiguedad',
        bin = alt.BinParams(step=6)
        ),
    y = 'cesados',
#    color = 'Producto'
).facet(
    'Producto',
    columns = 1
).resolve_scale(
    x='independent',
    y='independent'
)

alt.FacetChart(...)

* filtro el dataset general para obtener el dataset que utilizare segun el analisis realizado

In [22]:
ceses['INGRESO'] = ceses['INGRESO'].to_numpy().astype('datetime64[M]')
ceses['antiguedad'] = (((ceses['CESE'] - ceses['INGRESO']).dt.days)/30).round(0)

ceses_asesores = ceses[(
    ceses['CARGO'].isin(
        ['ASESOR NEGOCIOS CREDITO GRUPAL',
        'ASESOR NEGOCIOS CREDITO INDIVIDUAL'])) &
    (ceses['TIPO'] == 'VOLUNTARIA') &
    (ceses['antiguedad'] >=3)]

### Joins


#### Jalo PAM para obtener
* El nro de pams historico por codigo
* al duracion el pam y aque mes del pam se van

In [26]:
EMDT = pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/EMDT.xlsx',
    usecols = 'C,Q,U,V,W',
    converters = {'CODIGO CFIS':str}
    ).rename(
        columns={
            'CODIGO CFIS':'cod',
            'MOTIVO PRINCIPAL':'pam_motivo',
            'FECHA DE INICIO DE EVALUACIÓN\n(dd/mm/aa)':'pam_fi',
            'FECHA DE CIERRE DE LA EVALUACIÓN\n(dd/mm/aa)':'pam_fc',
            'STATUS':'pam_status'}
    )

In [67]:
#Total pams por COD
pam_nro = EMDT.groupby(
    ['cod'],
    as_index = False).agg(pam_total = ('cod', 'count'))

In [68]:
# Duracion promedio PAM por COD
EMDT['pam_duracion'] = round(((
                        EMDT['pam_fc']-
                        EMDT['pam_fi']).dt.days
                        )/30,0)

pam_duracion = EMDT.groupby(
                    ['cod'],
                    as_index=False).agg(
                        pam_duracion = ('pam_duracion', 'mean')).round(0)

In [69]:
# Dummies de PAM por COD
pam_dummy = pd.get_dummies(
    EMDT,
    columns = [
        'pam_motivo',
        'pam_status'])
        
pam_dummy = pam_dummy.groupby([
    'cod'],
    as_index=False)[
    'pam_motivo_DESEMPEÑO',
    'pam_motivo_DISCIPLINA',
    'pam_motivo_MIXTO',
    'pam_status_ANULADO',
    'pam_status_EN PROCESO',
    'pam_status_FINALIZADO EXITOSO',
    'pam_status_FINALIZADO NO EXITOSO',
    'pam_status_FINALIZADO POR CESE',
    'pam_status_NO ACEPTA INGRESAR AL PROGRAMA',
    'pam_status_SUSPENDIDO'].sum()



C:\Users\Anna\AppData\Local\Temp/ipykernel_14292/641652520.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pam_dummy = pam_dummy.groupby([


In [74]:
# uniendo todo PAM por codigo y borrando la col cod que se duplica
pam = pd.concat([pam_nro,pam_duracion,pam_dummy], axis=1)
pam = pam.loc[:,~pam.columns.duplicated()]

#### Jalo Vacaciones para obtener
* dias de vacaciones pendientes en su ultima fecha activo

In [95]:
vacaciones = pd.read_excel(
  'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/Vacaciones.xlsx',
  usecols='A,B,AC',
  converters={'COD':str}).rename(
  columns={
    'COD':'cod',
    'MES':'mes_vacas',
    'SALDO TOTAL DIAS\n(sin Negativos)':'vacas_pend'
  }  
) 

In [118]:
vacas_max = vacaciones.groupby(
    by=['cod'],
    as_index=False).agg(
        mes_vacas = ('mes_vacas','max'))
        
vacas_pend = pd.merge(
    vacas_max,
    vacaciones,
    left_on = ['cod','mes_vacas'],
    right_on = ['cod','mes_vacas'])

In [ ]:
# Bases que necesitare
memos_ap = pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/Memos y Ceses RRLL.xlsx',
    usecols='E,H:P,S,T,W:Z') #nro, falta, sanciones, tiempo antes de memo

DM = pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/Descansos Medicos y Subsidios.xlsx',
    usecols='E,H:P,S,T,W:Z') #nro, dias total dm

CAP_p = pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/CAP y PPTO.xlsx',
    usecols='E,H:P,S,T,W:Z') #cap de la agecia

recas = pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/Recategorizaciones CG CI.xlsx',
    usecols='E,H:P,S,T,W:Z') #nro recas # tiempo de reca

rvci =pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/RV CI.xlsx',
    usecols='E,H:P,S,T,W:Z') #monto desemb rv generado

rvcg =pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/RV CG.xlsx',
    usecols='E,H:P,S,T,W:Z') #monto desemb rv generado

desemp =pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/desempeño 2021.xlsx',
    usecols='E,H:P,S,T,W:Z') #promedio desemp

reconocimiento = pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/Reconocimiento.xlsx',
    usecols='E,H:P,S,T,W:Z') #nro reco, monto premio

hijxs =pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/Hijos por colab.xlsx',
    usecols='E,H:P,S,T,W:Z') #total hijos colab, edad prom hijxs
    
gptw =pd.read_excel(
    'C:/Users/Anna/Documents/1. Compartamos Financiera/Database/GPTW detalle.xlsx',
    usecols='E,H:P,S,T,W:Z') #prom gptw agencia 


### Individual

In [28]:
ceses.dtypes

CODIGO                              int64
CATEGORIA A.                       object
CARGO                              object
DEPARTAMENTO / GERENCIA            object
GERENCIA CENTRAL                   object
AGENCIA                            object
TERRITORIO                         object
DIVISION                           object
INGRESO                    datetime64[ns]
CESE                       datetime64[ns]
Total o Temprana                   object
Producto                           object
TIPO                               object
MOTIVO PRIMARIO                    object
Desgloce                           object
DESCRIPCION DEL CESE               object
dtype: object

In [27]:
ceses.isnull().sum()

CODIGO                        0
CATEGORIA A.               2011
CARGO                         0
DEPARTAMENTO / GERENCIA       0
GERENCIA CENTRAL              0
AGENCIA                       0
TERRITORIO                    0
DIVISION                      0
INGRESO                       0
CESE                          0
Total o Temprana              0
Producto                      0
TIPO                          0
MOTIVO PRIMARIO               0
Desgloce                      0
DESCRIPCION DEL CESE         96
dtype: int64